# Background subtraction using top-hat in scikit-image and pyclesperanto
This notebook compares different implementations of a background subtraction method.

**Note:** benchmarking results vary heavily depending on image size, kernel size, used operations, parameters and used hardware. Use this notebook to adapt it to your use-case scenario and benchmark on your target hardware. If you have different scenarios or use-cases, you are very welcome to submit your notebook as pull-request!

In [1]:
import pyclesperanto_prototype as cle
import pyclesperanto as pcle

from skimage import morphology
import time

# to measure kernel execution duration properly, we need to set this flag. It will slow down exection of workflows a bit though
cle.set_wait_for_kernel_finish(True)

# selet a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device('RTX')

<NVIDIA GeForce RTX 4090 on Platform: NVIDIA CUDA (1 refs)>

In [2]:
pcle.set_wait_for_kernel_to_finish(True)
pcle.select_device('RTX')

(OpenCL) NVIDIA GeForce RTX 4090 (OpenCL 3.0 CUDA)
	Type: GPU
	Compute Units: 128
	Global Memory Size: 25358 MB
	Maximum Object Size: 6339 MB

In [3]:
radius = 10
disk_kernel = morphology.ball(radius)
square_kernel = morphology.cube(radius)

In [4]:
# test data
import numpy as np

test_image = np.random.random([50, 1024, 1024]).astype(np.uint8)

In [5]:
# top-hat (disk) with pyclesperanto_prototype
result_image = None

test_image_gpu = cle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = cle.top_hat_sphere(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto_prototype top-hat-shere duration: " + str(time.time() - start_time))

pyclesperanto_prototype top-hat-shere duration: 0.07216572761535645
pyclesperanto_prototype top-hat-shere duration: 0.06630992889404297
pyclesperanto_prototype top-hat-shere duration: 0.0659325122833252
pyclesperanto_prototype top-hat-shere duration: 0.06356620788574219
pyclesperanto_prototype top-hat-shere duration: 0.06781673431396484


In [6]:
# top-hat (disk) with pyclesperanto
result_image = None

test_image_gpu = pcle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = pcle.top_hat(test_image_gpu, result_image, radius_x=radius, radius_y=radius, connectivity="sphere")
    print("pyclesperanto top-hat-shere duration: " + str(time.time() - start_time))

pyclesperanto top-hat-shere duration: 0.06919169425964355
pyclesperanto top-hat-shere duration: 0.06446290016174316
pyclesperanto top-hat-shere duration: 0.06555461883544922
pyclesperanto top-hat-shere duration: 0.06267595291137695
pyclesperanto top-hat-shere duration: 0.06412076950073242


In [7]:
# top-hat (square) using pyclesperanto_prototype
result_image = None

test_image_gpu = cle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = cle.top_hat_box(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto_prototype top-hat-box duration: " + str(time.time() - start_time))

pyclesperanto_prototype top-hat-box duration: 0.02030634880065918
pyclesperanto_prototype top-hat-box duration: 0.02023458480834961
pyclesperanto_prototype top-hat-box duration: 0.02082657814025879
pyclesperanto_prototype top-hat-box duration: 0.020810842514038086
pyclesperanto_prototype top-hat-box duration: 0.02071523666381836


In [8]:
# top-hat (square) using pyclesperanto
result_image = None

test_image_gpu = pcle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = pcle.top_hat(test_image_gpu, result_image, radius_x=radius, radius_y=radius, connectivity="box")
    print("pyclesperanto top-hat-box duration: " + str(time.time() - start_time))

pyclesperanto top-hat-box duration: 0.01052713394165039
pyclesperanto top-hat-box duration: 0.009463071823120117
pyclesperanto top-hat-box duration: 0.009361982345581055
pyclesperanto top-hat-box duration: 0.009383201599121094
pyclesperanto top-hat-box duration: 0.010474681854248047


In [9]:
# top-hat (disk) with scikit-image
result_image = None

for i in range(0, 5):
    start_time = time.time()
    result_image = morphology.white_tophat(test_image, footprint=disk_kernel)
    print("skimage top-hat disk duration: " + str(time.time() - start_time))

skimage top-hat disk duration: 227.64173436164856
skimage top-hat disk duration: 226.5375473499298
skimage top-hat disk duration: 226.06916117668152
skimage top-hat disk duration: 224.93067979812622
skimage top-hat disk duration: 224.46945238113403


In [10]:
# top-hat (square) with scikit-image`
result_image = None

for i in range(0, 5):
    start_time = time.time()
    result_image = morphology.white_tophat(test_image, footprint=square_kernel)
    print("skimage top-hat square duration: " + str(time.time() - start_time))

skimage top-hat square duration: 1.3502135276794434
skimage top-hat square duration: 1.3517491817474365
skimage top-hat square duration: 1.373570442199707
skimage top-hat square duration: 1.3558220863342285
skimage top-hat square duration: 1.373267412185669


### Impact of `dtype` on speed

In [14]:
test_image = np.random.random([50, 1024, 1024]).astype(np.float32)

In [15]:
# top-hat (square) using prototype
result_image = None

test_image_gpu = cle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = cle.top_hat_box(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto_prototype top-hat-box duration: " + str(time.time() - start_time))

pyclesperanto_prototype top-hat-box duration: 0.01829671859741211
pyclesperanto_prototype top-hat-box duration: 0.020997285842895508
pyclesperanto_prototype top-hat-box duration: 0.020043611526489258
pyclesperanto_prototype top-hat-box duration: 0.019987821578979492
pyclesperanto_prototype top-hat-box duration: 0.020021915435791016


In [16]:
# top-hat (square) using pyclesperanto
result_image = None

test_image_gpu = pcle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = pcle.top_hat(test_image_gpu, result_image, radius_x=radius, radius_y=radius, connectivity="box")
    print("pyclesperanto top-hat-box duration: " + str(time.time() - start_time))

pyclesperanto top-hat-box duration: 0.017570018768310547
pyclesperanto top-hat-box duration: 0.018915653228759766
pyclesperanto top-hat-box duration: 0.019094467163085938
pyclesperanto top-hat-box duration: 0.017920494079589844
pyclesperanto top-hat-box duration: 0.017951011657714844
